In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import IPython
import tensorflow as tf
from keras.layers import Conv2D, Conv2DTranspose
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import load_img
import tensorflow_datasets as tfds
import pathlib
import numpy as np
from skimage import io
from skimage.util import random_noise
import os
import matplotlib.pyplot as plt
import random

In [3]:
batch_size     =  5
image_size     =  512
INFERENCE_SIZE =  224
epochs         =  3
fit_verbosity  =  1
main_dir       = 'drive/MyDrive/datascience/dataset_1/Photo/'

In [4]:
all_image_paths = [main_dir+file for file in os.listdir(main_dir) if file.endswith('.jpg')]

print('Total number of images:', len(all_image_paths))

Total number of images: 10000


In [5]:
train_image_paths = all_image_paths[:9000]
test_image_paths  = all_image_paths[9000:]

print('Total number of train:', len(train_image_paths))
print('Total number of test:', len(test_image_paths))

Total number of train: 9000
Total number of test: 1000


In [6]:
def load_images(paths, size=image_size):
    images = []
    for path in paths:
        image = load_img(path, target_size=(size, size, 3))
        image = np.array(image)/255.0 # Normalize image pixel values to be between 0 and 1
        images.append(image)
    return np.array(images)

In [7]:
def add_noise(images):
    noise  = random_noise(images, mode='gaussian', mean=0, var=0.05)
    #noise  = random_noise(noise, mode='s&p',      amount=0.2, salt_vs_pepper=0.5)
    return noise

In [8]:
import matplotlib.pyplot as plt
#Show images
def show_images(X, n):
    plt.figure(figsize=(30, 30))
  
    for i in range(n):
        ax = plt.subplot(6, 6, i+1)
        img = X[i]
        plt.imshow(img)
        plt.axis("off")
    plt.show()

In [9]:
def datagen(paths, size=image_size, batch_size=5):
    for x in range(0, len(paths), batch_size):
        batch_paths  = paths[x:x+batch_size]
        # print(batch_paths)
        batch_images = load_images(batch_paths, size=size)
        noise_images = add_noise(batch_images)
        yield noise_images, batch_images

In [10]:
# train_dataset = datagen(train_image_paths, size=image_size, batch_size=batch_size)
# test_dataset  = datagen(test_image_paths, size=image_size, batch_size=batch_size)

In [11]:
# first_batch       = next(train_dataset)

# print("Original : ")
# show_images(first_batch[0], n=5)
# print("Noisy : ")
# show_images(first_batch[1], n=5)

In [12]:
image = Input(shape=(None,None,3))

# Encoder
l1 = Conv2D(64, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(image)     
l2 = Conv2D(64, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l1)

l3 = MaxPooling2D(padding='same')(l2)
l3 = Dropout(0.3)(l3)
l4 = Conv2D(128, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l3)
l5 = Conv2D(128, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l4)

l6 = MaxPooling2D(padding='same')(l5)
l7 = Conv2D(256, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l6)

#Decoder
l8 = UpSampling2D()(l7)
l9 = Conv2D(128, (3,3), padding='same', activation='relu',
           activity_regularizer=regularizers.l1(10e-10))(l8)
l10 = Conv2D(128, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l9)

l11 = add([l5,l10])
l12 = UpSampling2D()(l11)
l13 = Conv2D(64, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l12)
l14 = Conv2D(64, (3,3), padding='same', activation='relu',
            activity_regularizer=regularizers.l1(10e-10))(l13)

l15 = add([l14,l2])

decoded = Conv2D(3, (3,3), padding='same', activation='relu',
                activity_regularizer=regularizers.l1(10e-10))(l15)
model = Model(image, decoded)

In [13]:
model.compile(optimizer=RMSprop(learning_rate=0.0001),
             loss='binary_crossentropy')

In [ ]:
# history = model.fit(train_dataset,
#                  batch_size      = batch_size,
#                  epochs          = epochs,
#                  verbose         = fit_verbosity,
#                  shuffle         = True,
#                  steps_per_epoch = steps
#                  )


print(batch_size)
print(epochs)
steps  = int(len(train_image_paths)/batch_size)
print(steps)
for epoch in range(epochs):
    model.fit(datagen(train_image_paths, size=image_size, batch_size=batch_size), epochs=1, steps_per_epoch=steps)

10
2
900
431/900 [=============>................] - ETA: 2:42:51 - loss: 0.5654

In [ ]:
steps = int(len(test_image_paths)/batch_size)
model.evaluate(test_dataset, steps=steps)

In [ ]:
model.save('drive/MyDrive/datascience/mymodel')

In [ ]:
def plot_results(noise_image, reconstructed_image, image):
    w = 15
    h = len(noise_image)*5
    fig = plt.figure(figsize=(w, h))
    columns = 3
    rows = len(noise_image)
    for i in range(1, rows*columns, columns):
        fig.add_subplot(rows, columns, i)
        plt.axis('off')
        plt.title('Image with noise')
        plt.imshow(noise_images[int((i-1)/columns)])
    
        fig.add_subplot(rows, columns, i+1)
        plt.axis('off')
        plt.title('Reconstructed Image')
        plt.imshow(reconstructed[int((i-1)/columns)])
        
        fig.add_subplot(rows, columns, i+2)
        plt.axis('off')
        plt.title('Original Image')
        plt.imshow(images[int((i-1)/columns)])
    
    plt.show()


paths          = random.sample(test_image_paths, batch_size)
images         = load_images(paths, size=image_size)
noise_images   = add_noise(images)

reconstructed  = model.predict(noise_images)

plot_results(noise_images, reconstructed, images)